Importing the BindingDB_Kd dataset

In [19]:
from tdc.multi_pred import DTI
bindingDB_data = DTI(name = 'BindingDB_Kd')

Found local copy...
Loading...
Done!


Cold split method implementation on Drug column.
Need to verify if column name attribute is required or not ?

In [2]:
bindingDB_cold_split = bindingDB_data.get_split(method = 'cold_split', column_name = 'Drug', seed = 42, frac = [0.7, 0.1, 0.2])

In [3]:
train_set = bindingDB_cold_split['train']

In [4]:
#train_set = train_set.head(10)
train_set

,Drug_ID,Drug,Target_ID,Target,Y
0,4316.0,COc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.490
1,4293.0,NS(=O)(=O)c1ccc(S(=O)(=O)NCc2cccs2)s1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.830
2,1612.0,COc1ccc(N2CC(O)c3cc(S(N)(=O)=O)sc3S2(=O)=O)cc1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.160
3,4369102.0,CCN[C@H]1CN(CCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.320
4,68844.0,CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.130
...,...,...,...,...,...
34696,76311094.0,O=C(NCCCCS(=O)(=O)c1ccccc1)c1ccc2nccn2c1,P43490,MNPAAEAEFNILLATDSYKVTHYKQYPPNTSKVYSYFECREKKTEN...,860.000
34697,57406853.0,CCOc1cc(C(C)(C)C)ccc1C1=N[C@@](C)(c2ccc(Cl)cc2...,NaN,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,2.900
34698,58573469.0,CC(C)[C@@H](CS(=O)(=O)C(C)C)N1C(=O)[C@@](C)(CC...,NaN,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,0.045
34699,113557.0,CCCCCCCOC1OC(CO)C(O)C(O)C1O,P08191,MKRVITLFAVLLMGWSVNAWSFACKTANGTAIPIGGGSANVYVNLA...,17.000


Apply fingerprint embedding from rdkit

In [5]:
from rdkit.Chem.Fingerprints import FingerprintMols


RDKit WARNING: [20:43:27] Enabling RDKit 2019.09.3 jupyter extensions


Added a new column 'Drug_vector to store the fingerprint embedded values applied on Drug column'

In [6]:
dataSource = train_set.drop(['Target_ID', 'Drug_ID', 'Y'], axis=1)
dataSource['Drug_vector'] = ''
dataSource

,Drug,Target,Drug_vector
0,COc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,
1,NS(=O)(=O)c1ccc(S(=O)(=O)NCc2cccs2)s1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,
2,COc1ccc(N2CC(O)c3cc(S(N)(=O)=O)sc3S2(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,
3,CCN[C@H]1CN(CCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,
4,CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,
...,...,...,...
34696,O=C(NCCCCS(=O)(=O)c1ccccc1)c1ccc2nccn2c1,MNPAAEAEFNILLATDSYKVTHYKQYPPNTSKVYSYFECREKKTEN...,
34697,CCOc1cc(C(C)(C)C)ccc1C1=N[C@@](C)(c2ccc(Cl)cc2...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,
34698,CC(C)[C@@H](CS(=O)(=O)C(C)C)N1C(=O)[C@@](C)(CC...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,
34699,CCCCCCCOC1OC(CO)C(O)C(O)C1O,MKRVITLFAVLLMGWSVNAWSFACKTANGTAIPIGGGSANVYVNLA...,


Reference taken from rdkit

In [7]:
from rdkit import Chem
import numpy as np
import tensorflow as tf
import torch

for ind, drug in enumerate(dataSource['Drug']):
    mol = Chem.MolFromSmiles(drug)
    fp = Chem.RDKFingerprint(mol)
    intmap = map(int, fp.ToBitString())
    dataSource['Drug_vector'][ind] = np.array(list(intmap))
    
    
dataSource

,Drug,Target,Drug_vector
0,COc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."
1,NS(=O)(=O)c1ccc(S(=O)(=O)NCc2cccs2)s1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."
2,COc1ccc(N2CC(O)c3cc(S(N)(=O)=O)sc3S2(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, ..."
3,CCN[C@H]1CN(CCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, ..."
4,CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, ..."
...,...,...,...
34696,O=C(NCCCCS(=O)(=O)c1ccccc1)c1ccc2nccn2c1,MNPAAEAEFNILLATDSYKVTHYKQYPPNTSKVYSYFECREKKTEN...,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
34697,CCOc1cc(C(C)(C)C)ccc1C1=N[C@@](C)(c2ccc(Cl)cc2...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,"[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, ..."
34698,CC(C)[C@@H](CS(=O)(=O)C(C)C)N1C(=O)[C@@](C)(CC...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,"[0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, ..."
34699,CCCCCCCOC1OC(CO)C(O)C(O)C1O,MKRVITLFAVLLMGWSVNAWSFACKTANGTAIPIGGGSANVYVNLA...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, ..."


Creating a copy is not required. Can be omitted later

In [8]:
dataSourceCopy = dataSource.copy(deep=True)

In [9]:
dataSourceCopy

,Drug,Target,Drug_vector
0,COc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."
1,NS(=O)(=O)c1ccc(S(=O)(=O)NCc2cccs2)s1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."
2,COc1ccc(N2CC(O)c3cc(S(N)(=O)=O)sc3S2(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, ..."
3,CCN[C@H]1CN(CCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, ..."
4,CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, ..."
...,...,...,...
34696,O=C(NCCCCS(=O)(=O)c1ccccc1)c1ccc2nccn2c1,MNPAAEAEFNILLATDSYKVTHYKQYPPNTSKVYSYFECREKKTEN...,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
34697,CCOc1cc(C(C)(C)C)ccc1C1=N[C@@](C)(c2ccc(Cl)cc2...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,"[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, ..."
34698,CC(C)[C@@H](CS(=O)(=O)C(C)C)N1C(=O)[C@@](C)(CC...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,"[0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, ..."
34699,CCCCCCCOC1OC(CO)C(O)C(O)C1O,MKRVITLFAVLLMGWSVNAWSFACKTANGTAIPIGGGSANVYVNLA...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, ..."


In [10]:
torch.tensor(dataSourceCopy['Drug_vector'][0])

tensor([0, 0, 1,  ..., 0, 1, 1])

In [11]:
#for ind in range(len(dataSourceCopy)):
    #print("ind: ", ind)
    #t = torch.tensor(dataSourceCopy['Drug_vector'][ind])
    #print("value", t)
    #dataSourceCopy['Drug_vector'][ind] = t

In [12]:
drug_vector = dataSourceCopy['Drug_vector']

In [13]:
dataSourceCopy['Drug_vector'][0]

array([0, 0, 1, ..., 0, 1, 1])

For now I have just worked on first batch of size 256 as I am not sure if the above implementation is correct.


In [14]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=drug_vector, shuffle=True, batch_size=256)

In [15]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [16]:
sample = next(iter(train_loader))


In [17]:
sample

tensor([[1, 0, 0,  ..., 0, 0, 1],
        [1, 1, 1,  ..., 0, 1, 0],
        [0, 1, 1,  ..., 0, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 0, 1],
        [1, 1, 0,  ..., 1, 0, 1],
        [1, 1, 1,  ..., 0, 1, 1]])

In [18]:

print("Start")

from sklearn.cluster import AgglomerativeClustering
#for linkage in ('ward', 'average', 'complete', 'single'):

for linkage in ('ward', 'average'):
    clustering = AgglomerativeClustering(linkage=linkage, n_clusters=50)
    clustering.fit(sample)
    print("Final clusters: ", linkage, ": ", clustering.labels_)

Start
Final clusters:  ward :  [44 39  3  0 25  0 16 29 33  0 12 34 15 33 24 31 28  1 26 45  1 10 13  9
 23 47 28  4 47 32 35 44 33  0  3  7 48 10  3 29 11 32 15 48  5 49 31 45
 17  4 15 20  5  5  1 15  5 27  0  0 40 16 42  5  6 25 38 21  3  6 43  9
 17  0  3  7  4 41  5  2 37 15 12  1  2 29 42 33 16 32 48  7  9  7 24 12
 42  7 24 22  2 33  7 48 14 24 30 16 26  8 16 31  8 11  5  5 42 39  0 15
 39 24  5 33 28  9 49  3  3  8  0 27 23  9 48 10 25  2 36 14 15  4 34  7
 14  3 30 28  9 12 29  2 49  7 46  7 33  4  9 37 22 16 43 18  1  1  9 24
 35 32 14 37 19 16 37  0 43 19 19 43 31  0 16  8  7 44 36 13 22  0 15 39
 16  2  7 15  1  0  0  6  9  0 23 18 27  6 17  7 13 36 45  9  2  9  7 39
  0 40 35  1  7 33 20 13 33 41 36 33 44 31 15 46 20 24 16 13 26 19 23  7
 33 35 12  7 15 45 11 38 19 34 21 43 10 11 17 23]
Final clusters:  average :  [23 46  4 13  8 13  1 13 16 13 17 18  2 16 14 16 18 10 15 18 10  0 23 27
  4  2 18  3  2 42 20 23 16 13  7  1 34  0  7 13 25 42 20 34 39  5 16 18
  4  3 43 11 29